In [2]:
import gymnasium as gym
import numpy as np

import time
from collections import defaultdict
from tqdm.auto import tqdm

/Users/davoodwadi/mambaforge/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
env = gym.make("LunarLander-v2", render_mode="human")
observation, info = env.reset()

In [4]:
env.action_space

Discrete(4)

In [5]:
action = env.action_space.sample()
action

0

In [6]:
observation, reward, terminated, truncated, info = env.step(action)
observation, reward, terminated, truncated, info

(array([-0.00536461,  1.4205514 , -0.27131712,  0.20117281,  0.00615575,
         0.06082705,  0.        ,  0.        ], dtype=float32),
 0.8613720056758041,
 False,
 False,
 {})

In [7]:
observation, info = env.reset()
observation, info

(array([-0.00206156,  1.4143422 , -0.2088244 ,  0.1520939 ,  0.00239557,
         0.04730177,  0.        ,  0.        ], dtype=float32),
 {})

In [8]:
env.close()

In [9]:
env = gym.make("LunarLander-v2", render_mode="human")

for _ in range(100):
    observation, info = env.reset()
    action = env.action_space.sample()
    observation, reward, terminated, truncated, info = env.step(action)

KeyboardInterrupt: 

In [ ]:
for _ in range(1000):
    action = env.action_space.sample()  # agent policy that uses the observation and info
    observation, reward, terminated, truncated, info = env.step(action)

    if terminated or truncated:
        observation, info = env.reset()

env.close()

# Blackjack

In [ ]:
env = gym.make("Blackjack-v1", sab=False, render_mode="human")

In [ ]:
obs, info = env.reset() # (21, 1, 1): player's sum, dealer's face up, useable ace (ace count as 11 w/o busting)
obs, info

In [69]:
env.action_space

Discrete(2)

In [70]:
env.action_space.sample()

1

In [95]:
for _ in range(2):
    action = env.action_space.sample()
    state, reward, terminated, truncated, info = env.step(action)
    if terminated or truncated:
        print(state[0])
        time.sleep(3)
        new_state, info = env.reset()

15


In [89]:
env.reset()

((9, 6, 0), {})

In [91]:
state, reward, terminated, truncated, info = env.step(action)
state, reward, terminated

((24, 6, 0), -1.0, True)

In [96]:
state, reward, terminated

((15, 1, 0), -1.0, True)

Folding

In [263]:
env.reset()

((17, 9, 0), {})

# Dealer
After the player sticks, the dealer reveals their facedown card, and draws cards until their sum is **17 or greater**. If the dealer goes bust, the player wins.

In [220]:
action = 0
state, reward, terminated, truncated, info = env.step(action)
state, reward, terminated

((15, 6, 0), -1.0, True)

In [207]:
action = 1
state, reward, terminated, truncated, info = env.step(action)
state, reward, terminated

((28, 10, 0), -1.0, True)

## policy iteration

### Starting State
The starting state is initialised in the following range.

| Observation | Min | Max |
| --- | --- | --- |
| Player current sum | 4 | 12 (sic) |
| Dealer showing card value | 2 | 11 |
| Usable Ace | 0 | 1 |



### initialization

In [99]:
env.observation_space

Tuple(Discrete(32), Discrete(11), Discrete(2))

In [136]:
env.observation_space[0].sample()

18

In [131]:
env.observation_space[1].sample()

2

In [139]:
env.observation_space[2].sample()

1

In [140]:
env.reward_range

(-inf, inf)

initialize all states to zero

In [264]:
v = np.zeros((32, 11, 2))
v.shape

(32, 11, 2)

In [265]:
policy = defaultdict(lambda : np.random.randint(2))

In [266]:
state = (1,1,1)

In [267]:
v[state]

0.0

In [192]:
from itertools import product

In [310]:
states = list(product(range(4, 32), range(2, 11), range(2)))
len(states)

504

### policy evaluation

In [268]:
from gymnasium.wrappers import TransformObservation

In [291]:
state = (2,10,0)
env = TransformObservation(env, lambda obs: state)
env.reset()

((2, 10, 0), {})

In [292]:
env.step(0)

((2, 10, 0), 1.0, True, False, {})

In [221]:
theta = 0.1
gamma = 1.0

In [320]:
delta = float('inf')
while delta > theta:
    delta = 0
    # policy evaluation
    for state in tqdm(states):
        # print(state)
        old_value = v[state]
        # set environment state to state
        env.reset()
        env = TransformObservation(env, lambda obs: state)
        # get action according to policy
        action = policy[state]
        # take action and receive next state and reward
        statePrime, reward, terminated, truncated, info = env.step(action)
        # value update using deterministic policy
        v[state] = reward + gamma*v[statePrime]

        # update delta
        delta = np.max([np.abs(old_value-v[state]), delta])
        # check convergence
        if delta < theta: break
    
    # policy improvement
    policy_stable = True
    for state in tqdm(states):
        old_action = policy[state]
        # set environment state to state
        env.reset()
        env = TransformObservation(env, lambda obs: state)

        # get new policy that maximizes the value function
        reference_value = float('-inf')
        best_action = -1
        for action in range(env.action_space.n):
            statePrime, reward, terminated, truncated, info = env.step(action)
            value_action = reward + gamma*v[statePrime]
            # find the action that maximizes the value function
            if value_action>=reference_value: 
                best_action = action
                reference_value = value_action
        # update the policy
        policy[state] = best_action
        # check whether policy is optimal
        if old_action!=best_action: policy_stable=False

    if policy_stable: break

    break

 17%|███████▎                                  | 88/504 [00:44<03:28,  1.99it/s]


KeyboardInterrupt: 

In [321]:
old_value, v[state]

(-1.0, -1.0)

In [322]:
state, statePrime

((8, 10, 0), (8, 9, 1))

In [323]:
reward, v[statePrime]

(-1.0, 0.0)

In [339]:
state, info = env.reset()
state

(8, 10, 0)

In [342]:
env.step(1)

((8, 10, 0), -1.0, True, False, {})

### policy iteration based on random state initialization

In [343]:
env = gym.make("Blackjack-v1", sab=False, render_mode="human")

In [344]:
policy = defaultdict(lambda : np.random.randint(2))
states = list(product(range(4, 32), range(2, 11), range(2)))
v = np.zeros((32, 11, 2))
len(states), v.shape

(504, (32, 11, 2))

In [385]:
theta = 0.1
gamma = 1.0
n_eval = 27 * 10 * 2
n_update = 27 * 10 * 2
n_eval

540

In [386]:
delta = float('inf')
for e in tqdm(range(n_eval)):
    delta = 0
    # policy evaluation

    # get deterministic states
    state, _ = env.reset(seed=e)

    old_value = v[state]
    # get action according to policy
    action = policy[state]
    # take action and receive next state and reward
    statePrime, reward, terminated, truncated, info = env.step(action)
    # value update using deterministic policy
    v[state] = reward + gamma*v[statePrime]

    # update delta
    delta = np.max([np.abs(old_value-v[state]), delta])
    # check convergence
    # if delta < theta: 
    #     print(f'evaluation converged')
    #     break
    
    # log
    if np.abs(old_value-v[state])>0:
        print(f'for state {state} value changed from {old_value} to {v[state]}')

# policy improvement
for u in tqdm(range(n_update)):
    policy_stable = True
    
    state, _ = env.reset(seed=u)
    old_action = policy[state]
    
    # get new policy that maximizes the value function
    reference_value = float('-inf')
    best_action = -1
    for action in range(env.action_space.n):
        # reset to deterministic state
        state, _ = env.reset(seed=u)
        # calculate action value
        statePrime, reward, terminated, truncated, info = env.step(action)
        value_action = reward + gamma*v[statePrime]
        # find the action that maximizes the value function
        if value_action>=reference_value: 
            best_action = action
            reference_value = value_action
            best_statePrime = statePrime
            best_reward = reward
            best_valuePrime = value_action
    
    # update the policy
    policy[state] = best_action
    # check whether policy is optimal
    if old_action!=best_action: 
        print(f'for state {state} best action changed to {best_action}, statePrime {best_statePrime}={best_valuePrime}, reward {best_reward}')
        policy_stable=False


    # if policy_stable: 
    #     print(f'improvement converged')
    #     break


  0%|▏                                          | 2/540 [00:00<03:38,  2.46it/s]

for state (20, 7, 0) value changed from 12.0 to 13.0


  1%|▎                                          | 4/540 [00:01<04:11,  2.13it/s]

for state (7, 10, 0) value changed from 3.0 to -1.0


  1%|▍                                          | 6/540 [00:02<04:20,  2.05it/s]

...

 99%|████████████████████████████████████████▋| 536/540 [11:12<00:05,  1.25s/it]

for state (7, 4, 0) best action changed to 1, statePrime (11, 4, 0)=1.0, reward 0.0


 99%|████████████████████████████████████████▊| 537/540 [11:13<00:03,  1.25s/it]

for state (9, 10, 0) best action changed to 0, statePrime (9, 10, 0)=0.0, reward 1.0


100%|█████████████████████████████████████████| 540/540 [11:17<00:00,  1.25s/it]


### evaluate the policy using random states

In [381]:
state, _ = env.reset()
state

(19, 7, 0)

In [10]:
policy[state]

NameError: name 'policy' is not defined